Since DeKalb isn't that big, we can take a less efficient approach. Read in all the applicable Excel files, merge, and THEN clean. This is not very memory efficient but fine for this.

In [1]:
import pandas as pd
import os

CURR_PATH = os.getcwd()
DATA_PATH = os.path.join(CURR_PATH, '..', '..', 'data', 'dekalb')
FORMAT_PATH = os.path.join(CURR_PATH, '..', '..', 'input', 'var_maps', 'dekalb', 'dekalb.xlsx')
# Need to renamme PARDAT for 2022
REL_FILES = ["ASMT", "PARCEL DATA", "LEGAL DATA", "OWNERSHIP DATA", "DWELLING DATA"]

In [2]:
_, subdirs, files  = next(os.walk(DATA_PATH))
subdirs = subdirs [1:] # skip first subdir bc it is not relevant

print(*subdirs, sep='\n')

DeKalb County Full File Aug 2021
DeKalb County Full File EOY 2020
DeKalb County Full File EOY2016
DeKAlb County Full File EOY2017
DeKalb County Full File EOY2019
DeKalb County Full File EOY2022
DeKalb County Full File Oct 2018
DeKalb County Full File2015


In [3]:
data_info = pd.read_excel(FORMAT_PATH)

In [14]:
data_info

,old_name,new_name,data_type,source_file,Unnamed: 4,notes
0,TAXYR,tax_year,NaN,NaN,NaN,NaN
1,PARID,parcel_id,NaN,all,NaN,NaN
2,CITYNAME,site_city,NaN,PARCEL DATA,NaN,NaN
3,ADRNO;ADRDIR;ADRSTR;ADRSUF;ZIP1,site_addr,NaN,PARCEL DATA,NaN,NaN
4,CLASS,site_class_parcel,NaN,PARCEL DATA,NaN,NaN
5,LUC,site_luc_parcel,NaN,PARCEL DATA,NaN,NaN
6,TAXDIST,tax_district,NaN,LEGAL DATA,NaN,NaN
7,OWN1,owner_name_1,NaN,OWNERSHIP DATA,NaN,NaN
8,OWN2,owner_name_2,NaN,OWNERSHIP DATA,NaN,NaN
9,ZIP1,owner_zip,NaN,OWNERSHIP DATA,NaN,NaN


In [46]:
data_info = data_info.dropna(subset=["old_name"])

data_info.where(
    ~data_info["old_name"].str.contains(";") |
    ~data_info["old_name"].str.contains("\+")
)

error: nothing to repeat at position 0

In [41]:
for row in data_info.itertuples():
    print(row.old_name)

TAXYR
PARID
CITYNAME
ADRNO;ADRDIR;ADRSTR;ADRSUF;ZIP1
CLASS
LUC
TAXDIST
OWN1
OWN2
ZIP1
ADRNO;ADRDIR;ADRSTR;ADRSUF
nan
nan
nan
FMV BLDG + FMV LAND
FMV LAND
FMV BLDG
nan
nan
YRBLT
nan
nan
GRADE
nan
SFLA


In [ ]:
# need a func that produces a list of relevant files
# need a func that takes instructions from old_name

# correct the source files
# separate out joint columns

# join within each year and then append years

# cobb
# need to select for variables as well
# join within years and append

In [37]:
path = '/home/User/Desktop/test/sample.txt'

# Split the path in head and tail pair 
head_tail = os.path.split(path)
head_tail

('/home/User/Desktop/test', 'sample.txt')

Issues: some lines are longer than others (more commas), and the headers have special characters

Solution: can skip header line and read in separately then strip

In [33]:
import re

def get_rel_cols(path: str, data_info: pd.DataFrame) -> list:
    f = path.split("\\")[-1].replace(".csv", "")
    return data_info.loc[data_info["source_file"] == f, "old_name"].values
    
def read_file(path: str) -> pd.DataFrame:
    print("Reading file: ", path)
    
    rel_cols = get_rel_cols(path, data_info)
    print(rel_cols)

    with open(path, 'r') as f:
        header = f.readline()
        full_len = len(f.readlines())
        
    header = re.sub(r'[^a-zA-Z0-9,]', '', header)
    
    data = pd.read_csv(
        path, 
        encoding='UTF-8',
        skiprows=1,
        usecols=rel_cols
    )
    
    data.columns = header.split(',')
    
    print(f"Skipped {full_len - len(data)} rows due to data corruption.")
    
    return data

In [34]:
for subdir in subdirs:
    curr_path = os.path.join(DATA_PATH, subdir)
    rel_path_files = [os.path.join(curr_path, f"{name}.csv") for name in REL_FILES]
    data = [read_file(f) for f in rel_path_files]

Reading file:  c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File Aug 2021\ASMT.csv
[]


ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements

In [10]:
COLS = ["PARID"]

def read_file(file_path: str, cols: list = None):
    print(f"Reading {file_path}...")
    try:
        return pd.read_csv(file_path + '.csv', usecols=cols, on_bad_lines='warn', encoding='latin1')
    except Exception as e:
        return None

def read_file_depr(file_path):
    print(f"Reading {file_path}...")
    # some lines have an extra comma
    # ParserError: Error tokenizing data. C error: Expected 24 fields in line 3120, saw 25
    # Decided to use use_cols instead
    with open(f"{file_path}.csv") as f:
        lines = f.readlines()
    
    expected_len = len(lines[0].split(','))
    lines = [line.replace(',\n', '\n') if len(line.split(',')) != expected_len else line for line in lines]
    
    new_file = f"{file_path}_adj.csv"
    with open(f"{file_path}_adj.csv", 'w') as f:
        f.writelines(lines)
        
    return pd.read_csv(new_file, on_bad_lines='warn', encoding='latin1')

data = []
for subdir in subdirs:
    curr_path = os.path.join(DATA_PATH, subdir)
    data.append([read_file(os.path.join(curr_path, f), COLS) for f in REL_FILES])

Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File Aug 2021\ASMT...
Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File Aug 2021\PARCEL DATA...
Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File Aug 2021\LEGAL DATA...
Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File Aug 2021\OWNERSHIP DATA...
Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File Aug 2021\DWELLING DATA...
Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File EOY 2020\ASMT...
Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full File EOY 2020\PARCEL DATA...
Reading c:\Users\Nick\Documents\code\ga-tax-assessment\src\misc\..\..\data\dekalb\DeKalb County Full